In [ ]:
CONTENT_CSV_FILE = '../source_data/export.csv'

In [ ]:
## Load content
import csv

content = []
with open(CONTENT_CSV_FILE, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        content.append(row)

content

In [ ]:
categories = set()
for item in content:
    categories.add(item['DsCompetition'])
categories_items = { category:i  for i, category in enumerate(categories) }
categories_items

In [ ]:
CATEGORY_HEADER = 'DELETE FROM public."Category"; INSERT INTO public."Category" ("Id", "Description") VALUES \n'
CATEGORY_ROW = "({id}, '{description}')"
CATEGORY_SQL_FILE = '../17_insert_category.sql'

In [ ]:
# Create category SQL
category_sql = CATEGORY_HEADER
category_sql += ',\n'.join( [ CATEGORY_ROW.format(id=id, description=category) for category, id in categories_items.items() ])
category_sql += ';'
category_sql

In [ ]:
# WRITE CATEGORY SQL
with open(CATEGORY_SQL_FILE, 'w') as f:
    f.write(category_sql)

In [ ]:
import datetime
import random

CONTENT_HEADER = """DELETE FROM public."Content"; 
INSERT INTO public."Content"( \n"Id", "CategoryId", "Description", "Duration", "StartDate", "EndData") 
VALUES \n"""
CONTENT_ROW = "({id}, '{category_id}', '{description}', {duration}, '{start_date}', '{end_date}')"
CONTENT_SQL_FILE = '../18_insert_content.sql'
FUTURE=datetime.date(2999,12,31)
ENDED_CONTENT=0.2

In [ ]:
content_items = []
today = datetime.date.today()

for i, item in enumerate(content):
    ## Random between 6 months and 5 years
    from_date = today - datetime.timedelta(days=random.randint(180,365*5))
    content_items.append(dict(
        id=i,
        category_id=categories_items[item['DsCompetition']],
        description=item['NaAsset'].replace("'", "")),
        duration=int(float(item['QtMinutesDuration'])*60),
        start_date=from_date,
        end_date=FUTURE
    ))
print('Content items: ', len(content_items))
content_items


In [ ]:
ended_content = random.sample(
    range(0,len(content_items)), 
    int((ENDED_CONTENT)*len(content_items))
)
len(ended_content)

In [ ]:
# For ended content, set end date in the past 6 months randomly
for i in ended_content:
    to_date = today - datetime.timedelta(days=random.randint(1,180))
    content_items[i]['end_date'] = to_date

# Count and confirm items with end_date not in the FUTURE
count = len(list( filter(lambda x : x['end_date'] != FUTURE , content_items) ))
print('Ended content: ', count)

In [ ]:
# Generate SQL
content_sql = CONTENT_HEADER
content_sql += ',\n'.join( [ CONTENT_ROW.format(**item) for item in content_items ])
content_sql += ';'
content_sql

In [ ]:
# Write SQL file
with open(CONTENT_SQL_FILE, 'w') as f:
    f.write(content_sql)